In [142]:
# import the necessary libraries
from bs4 import BeautifulSoup
import requests
import json
import re
import datetime

In [143]:
# this function handles the situation when the html is embedded within javascript
def javascript_html_parse(source_arg):
	"""Return the JSON object as a dictionary"""
	start = "<script>window.__PRELOADED_STATE__ = "
	end = "</script"
	x = source_arg[source_arg.find(start)+len(start):]
	x = x[:x.find(end)]
	x = json.loads(x)
	return x

In [144]:
# Obtain all of the URL's that need to be scraped from 
# This is the first URL we start at, here we are collecting all of the URL's to be scraped from into 
# local area URL's 
url = "https://www.rightmove.co.uk/house-prices-in-London.html"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
b = soup.find_all("a", class_="head")
london_urls = []
for i in b:
	london_urls.append(i["href"])

borough_urls = []
for url in london_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		borough_urls.append(i["href"])

local_area_urls = []
for url in borough_urls:
	source = requests.get(url).text
	soup = BeautifulSoup(source, 'lxml')
	b = soup.find_all("a", class_="head")
	for i in b:
		local_area_urls.append(i["href"])	

In [145]:
# iterate over all of the URL's and scrape from them, and store the scraped data into a variable called houses
houses = {}
for area_url in local_area_urls[:2]:

	# while loop that obtains the address, property type, last sold price, last sold date, and its url (if it has one) from each house. 
	# Stores each value into a list. This is from the web page which shows different houses from a particular area.

	page_number = 1
	addresses, property_type, last_sold_price, last_sold_date, url = [], [], [], [], []
	#bush_hill_url = "https://www.rightmove.co.uk/house-prices/bush-hill-park.html"
	# obtain the number of pages to use in the while loop condition
	source = requests.get(area_url).text
	a = javascript_html_parse(source)
	num_pages = a["pagination"]["last"]
	#houses = {}
	while page_number <= num_pages:

		if page_number == 1:
			source = requests.get(area_url).text
		else:
			source = requests.get(area_url+f"?page={page_number}").text

		# all we want to extract is the address, the price & date of the last sale, and the URL.
		x = javascript_html_parse(source)
		for house in x["results"]["properties"]:
			addresses.append(house["address"])
			property_type.append(house["propertyType"])
			last_sold_price.append(house["transactions"][0]["displayPrice"])
			last_sold_date.append(house["transactions"][0]["dateSold"])
			url.append(house["detailUrl"])



		page_number += 1


	# Create a dictionary with all of the scraped info stored in it so far
	#houses = {}
	for i in range(len(addresses)):
		houses[addresses[i]] = {"property_type":property_type[i], "price":last_sold_price[i], "date":last_sold_date[i], "url": url[i]}




	for v in houses.values():
		if v['url'] == '':
			continue
		else:
			source = requests.get(v['url']).text
			soup = BeautifulSoup(source, 'lxml')
			script = soup.find("script", {"type":"text/javascript"}).text
			# Regex used to convert the JSON data structure into a python dict 
			# Use regex to extract json data from the script text
			json_script=re.findall(("(?s)(?<=window.PAGE_MODEL = )(.*$)"), script)[0]
			# Transforming json data within string into dictionary
			json_dict=json.loads(json_script)

			# The bullet pointed features # 
			try:
				features = json_dict["soldPropertyData"]["property"]["keyFeatures"]
			except TypeError: 
				pass
			else:
				v["features"] = features
			# got the floorplan URL
			try:
				floorplan_url = json_dict["soldPropertyData"]["property"]["floorplans"][0]['url']
			except Exception:
				# seen a TypeError and an IndexError
				pass
			else:
				v["floorplan_url"] = floorplan_url	
			# Scrape the pictures of the house ## commented this out for now, as I'm not using the images
			#house_image_urls = []
			#for image in json_dict["soldPropertyData"]["property"]["images"]:
			#	house_image_urls.append(image['url'])
			#v["house_image_urls"] = house_image_urls
			# location of the house
			latitude = json_dict["soldPropertyData"]["property"]["location"]["latitude"]
			longitude = json_dict["soldPropertyData"]["property"]["location"]["longitude"]
			v["latitude"] = latitude
			v["longitude"] = longitude
			# proximity to the stations
			station_names, distances = [], []
			for station in json_dict["soldPropertyData"]["property"]["nearestStations"]:
				station_names.append(station["name"])
				distances.append(station["distance"])
			station_proximities = {station:distance for station, distance in zip(station_names, distances)}
			v["station_proximities"] = station_proximities
			# size of the property
			units, sizes = [], []
			for i in json_dict["soldPropertyData"]["property"]["sizings"]:
				units.append(i['unit'])
				sizes.append(i['maximumSize'])
			dimensions = {unit:size for unit, size in zip(units, sizes)}
			v["property_size"] = dimensions

			# number of bedrooms and bathrooms
			bedrooms = json_dict["soldPropertyData"]["property"]['bedrooms']
			bathrooms = json_dict["soldPropertyData"]["property"]['bathrooms']
			v["bedrooms"] = bedrooms
			v["bathrooms"] = bathrooms

			# Whether the property is a new build or not
			new_build = json_dict["soldPropertyData"]["transactions"][0]['newBuild']
			v["new_build"] = new_build


In [146]:
houses

{'105, Canonsleigh Road, Dagenham, Greater London RM9 4DH': {'property_type': 'Terraced',
  'price': '£390,000',
  'date': '16 May 2022',
  'url': ''},
 '59, Bateman Close, Barking, Greater London IG11 8QS': {'property_type': 'Flat',
  'price': '£225,000',
  'date': '12 May 2022',
  'url': 'https://www.rightmove.co.uk/house-prices/details/england-123105338-14974252?s=791fe9e857a8dea92f2d04177af0a010ed90647d8449404cd8f3099af0e6d49f',
  'features': [],
  'floorplan_url': 'https://media.rightmove.co.uk/60k/59646/123105338/59646_CW0434_FLP_00_0000.jpeg',
  'latitude': 51.54126,
  'longitude': 0.08243,
  'station_proximities': {'Barking Station': 0.14006575393753187,
   'Upney Station': 0.8457992920654402,
   'Ilford Station': 1.3208233936340128},
  'property_size': {'ac': 0.01, 'ha': 0, 'sqft': 506, 'sqm': 47},
  'bedrooms': 1,
  'bathrooms': 1,
  'new_build': False},
 '54, Greenslade Road, Barking, Greater London IG11 9XE': {'property_type': 'Flat',
  'price': '£241,000',
  'date': '11 Ma

In [147]:
# change price from a string into an int
def price_int(x):
    a = x[1:]
    b = a.replace(",", "")
    b = int(b)
    return b

for features in houses.values():
    y = features['price']
    z = price_int(y)
    features['price'] = z
    

In [148]:
houses

{'105, Canonsleigh Road, Dagenham, Greater London RM9 4DH': {'property_type': 'Terraced',
  'price': 390000,
  'date': '16 May 2022',
  'url': ''},
 '59, Bateman Close, Barking, Greater London IG11 8QS': {'property_type': 'Flat',
  'price': 225000,
  'date': '12 May 2022',
  'url': 'https://www.rightmove.co.uk/house-prices/details/england-123105338-14974252?s=791fe9e857a8dea92f2d04177af0a010ed90647d8449404cd8f3099af0e6d49f',
  'features': [],
  'floorplan_url': 'https://media.rightmove.co.uk/60k/59646/123105338/59646_CW0434_FLP_00_0000.jpeg',
  'latitude': 51.54126,
  'longitude': 0.08243,
  'station_proximities': {'Barking Station': 0.14006575393753187,
   'Upney Station': 0.8457992920654402,
   'Ilford Station': 1.3208233936340128},
  'property_size': {'ac': 0.01, 'ha': 0, 'sqft': 506, 'sqm': 47},
  'bedrooms': 1,
  'bathrooms': 1,
  'new_build': False},
 '54, Greenslade Road, Barking, Greater London IG11 9XE': {'property_type': 'Flat',
  'price': 241000,
  'date': '11 May 2022',
  '

In [10]:
# Change the format of the date

In [149]:
# OCR
# Import the necessary libraries
# import all of the necessary libraries
import easyocr
import cv2

In [150]:
image1 = "https://media.rightmove.co.uk/72k/71134/111198221/71134_30837396_FLP_00_0002.jpeg"
reader = easyocr.Reader(['en'], gpu=False)
result = reader.readtext(image1)
result

Using CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████| 107.2% Complete

[([[4, 70], [262, 70], [262, 124], [4, 124]],
  '13th Floor',
  0.9569405236450418),
 ([[469, 97], [489, 97], [489, 119], [469, 119]], 'w', 0.5976982000865974),
 ([[284, 320], [372, 320], [372, 344], [284, 344]],
  '"Bedroom',
  0.7574518358776843),
 ([[279, 341], [383, 341], [383, 359], [279, 359]],
  '0.0Om * 4.78m1',
  0.4299373247246344),
 ([[325, 353], [367, 353], [367, 373], [325, 373]],
  '15*8")',
  0.5845066549310215),
 ([[394, 504], [478, 504], [478, 528], [394, 528]],
  'Bedroom',
  0.9999056945221165),
 ([[385, 523], [489, 523], [489, 543], [385, 543]],
  '3.19m * 2.70m',
  0.6102979585241233),
 ([[389, 539], [483, 539], [483, 557], [389, 557]],
  '(10\'6" x 8\'10")',
  0.7197790498556401),
 ([[392, 702], [478, 702], [478, 726], [392, 726]],
  'Bedroom',
  0.9898320707542342),
 ([[383, 723], [487, 723], [487, 741], [383, 741]],
  '3.18m * 2.70m',
  0.613414440517468),
 ([[391, 737], [483, 737], [483, 755], [391, 755]],
  '(10\'5" x 8\'10")',
  0.4137179057800107),
 ([[355, 

In [151]:
image_1 = 'https://media.rightmove.co.uk/72k/71134/116482163/71134_31111584_FLP_00_0000.jpeg'
reader1 = easyocr.Reader(['en'], gpu=False)
result1 = reader.readtext(image_1)
result1

Using CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████| 106.1% Complete

[([[135, 0], [399, 0], [399, 43], [135, 43]],
  'Second Floor',
  0.9994564296643119),
 ([[1026, 156], [1258, 156], [1258, 204], [1026, 204]],
  'Third Floor',
  0.9550033073947181),
 ([[206, 176], [356, 176], [356, 231], [206, 231]],
  'Balcony',
  0.7510318448595686),
 ([[1145, 460], [1312, 460], [1312, 509], [1145, 509]],
  'Bedroom',
  0.9999826981111722),
 ([[1129, 501], [1333, 501], [1333, 539], [1129, 539]],
  '3.77m x 2.58m',
  0.6104654561744267),
 ([[188, 518], [374, 518], [374, 568], [188, 568]],
  'Reception',
  0.9999980131775206),
 ([[1147, 531], [1319, 531], [1319, 571], [1147, 571]],
  '(12\'4" x 8\'6")',
  0.8324985893054271),
 ([[231, 559], [338, 559], [338, 602], [231, 602]],
  'Room',
  0.9999909996986389),
 ([[182, 598], [384, 598], [384, 630], [182, 630]],
  '4.93m X 4.04m',
  0.6257794497852901),
 ([[191, 627], [379, 627], [379, 663], [191, 663]],
  '(162" x 133")',
  0.4824622770668281),
 ([[171, 969], [291, 969], [291, 1005], [171, 1005]],
  'Kitchen',
  0.8913

In [152]:
scraped_text = ''
for i in result1:
    scraped_text += i[1]
    scraped_text += ","
    
text = scraped_text.split(",")
###text
        

In [153]:
scraped_text1 = ''
for i in result:
    scraped_text1 += i[1]
    scraped_text1 += ","
    
text1 = scraped_text1.split(",")
text1

['13th Floor',
 'w',
 '"Bedroom',
 '0.0Om * 4.78m1',
 '15*8")',
 'Bedroom',
 '3.19m * 2.70m',
 '(10\'6" x 8\'10")',
 'Bedroom',
 '3.18m * 2.70m',
 '(10\'5" x 8\'10")',
 'Reception',
 'Room',
 '7.98m X 3.77m',
 '(262" x 124")',
 'Kitchen',
 '93m X 2.Jem',
 '(190x7271',
 'Balcony',
 'Fire',
 'Escape',
 'Total area: approx. 101.1 sq. metres (1087.9 sq. feet)',
 '']

In [61]:
m_pattern = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
f_pattern = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|feet2|ft2|f2)", re.IGNORECASE)
for x in text1:
    for match in re.finditer(m_pattern, x):
        print('%s : %s' % (match.groups()))
    for match in re.finditer(f_pattern, x):
        print('%s : %s' % (match.groups())) 
        


101.1 : sq. m
1087.9 : sq. f


In [154]:
def size_floorplan(floorplan):
    """Obtain the size data from the floorplan"""
    reader = easyocr.Reader(['en']) # GPU=false taken out for now
    result = reader.readtext(floorplan)
    
    scraped_text = ''
    for i in result:
        scraped_text += i[1]
        scraped_text += ","
    text_list = scraped_text.split(",")
    
    m_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square meters|square metres|square meter|square metre|square m|square mt|sqmt|sq mt|sq.mt|sq. mt|sqm|sq m|sq.m|sq. m|meters2|metres2|meter2|metre2|mt2|m2)", re.IGNORECASE)
    f_patterns = re.compile(r"[+-]? *((?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)\s*(square feet|square ft|square f|sqft|sq ft|sq.ft|sq. ft|sqf|sq f|sq.f|sq. f|feet2|ft2|f2)", re.IGNORECASE)
    house_size = []
    for x in text_list:
        for match in re.finditer(m_patterns, x):
            #print('%s : %s' % (match.groups()))
            house_size.append(list(match.groups()))
        for match in re.finditer(f_patterns, x):
            #print('%s : %s' % (match.groups())) 
            house_size.append(list(match.groups()))
    for a in house_size:
        try:
            a[0] = int(a[0])
        except ValueError:
            a[0] = float(a[0])
            
    # then here you could find the
    house_size = dict((house_size))
    return house_size
    

In [155]:
r = size_floorplan('https://media.rightmove.co.uk/77k/76792/48779142/76792_HAM547-t-3762_FLP_01_0000.jpg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |█████████████████████████████████████████████████████| 107.5% Complete

In [156]:
# Code to take the largest returned value from 
amount = 0
for size in r.keys():
    if size > amount:
        amount = size
largest_size = {}
largest_size[amount] = r[amount]
largest_size[amount] = largest_size[amount].lower()
print(largest_size)

metre_values = ["square meters", "square metres", "square meter", "square metre", "square m", "square mt", "sqmt", "sq mt",
                "sq.mt", "sq. mt", "sqm", "sq m", "sq.m", "sq. m", "meters2", "metres2", "meter2", "metre2", "mt2", "m2"]
feet_values = ["square feet","square ft","square f","sqft","sq ft","sq.ft","sq. ft","sqf","sq f","sq.f","sq. f","feet2",
               "ft2","f2"]


if largest_size[amount] in metre_values:
    largest_size[amount] = "sq m"
elif largest_size[amount] in feet_values:
    largest_size[amount] = "sq ft"
print(largest_size)

{1190: 'sq ft'}
{1190: 'sq ft'}


In [158]:
import geopy.distance

In [161]:
outstanding_schools = [[51.4974948, -0.1356583],
                       [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.5345448, -0.2043853],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5906113, -0.1109709],
 [51.5202607, -0.0293396],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5436387, -0.0553621],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.5886383, -0.0117625],
 [51.5132537, -0.3043136],
 [51.5906113, -0.1109709],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5672808, -0.2710568],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4933675, 0.0098214],
 [51.5886121, 0.0823982],
 [51.5886121, 0.0823982],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4935082, -0.1178424],
 [51.5886121, 0.0823982],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5132537, -0.3043136],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5886121, 0.0823982],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5123443, -0.0909852],
 [51.538621, -0.1028346],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.5886383, -0.0117625],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.502781, -0.087738],
 [51.4935082, -0.1178424],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.502781, -0.087738],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.4933675, 0.0098214],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.502781, -0.087738],
 [51.4974948, -0.1356583],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.4611509, -0.0073177],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.6522994, -0.0807119],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.5436387, -0.0553621],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.5540666, 0.134017],
 [51.461311, -0.303742],
 [51.502781, -0.087738],
 [51.5672808, -0.2710568],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5886383, -0.0117625],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.577924, 0.2120829],
 [51.4990805, -0.1938253],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.4990156, -0.22915],
 [51.5255162, 0.0352163],
 [51.502781, -0.087738],
 [51.5255162, 0.0352163],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.5436387, -0.0553621],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4097742, -0.2108084],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.41233, -0.300689],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.502781, -0.087738],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.5202607, -0.0293396],
 [51.5390261, -0.1425516],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.5540666, 0.134017],
 [51.4974948, -0.1356583],
 [51.4933675, 0.0098214],
 [51.5540666, 0.134017],
 [51.3769529, -0.0956895],
 [51.5351832, -0.4481378],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.5886121, 0.0823982],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.6522994, -0.0807119],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4097742, -0.2108084],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.4611509, -0.0073177],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.4935082, -0.1178424],
 [51.4611509, -0.0073177],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.4935082, -0.1178424],
 [51.5906113, -0.1109709],
 [51.41233, -0.300689],
 [51.5906113, -0.1109709],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.4935082, -0.1178424],
 [51.5132537, -0.3043136],
 [51.6569225, -0.1949252],
 [51.4097742, -0.2108084],
 [51.5906113, -0.1109709],
 [51.6569225, -0.1949252],
 [51.5886383, -0.0117625],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4609218, -0.373149],
 [51.577924, 0.2120829],
 [51.5351832, -0.4481378],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.5672808, -0.2710568],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5351832, -0.4481378],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.6569225, -0.1949252],
 [51.3769529, -0.0956895],
 [51.439933, 0.154327],
 [51.6569225, -0.1949252],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.4609218, -0.373149],
 [51.5436387, -0.0553621],
 [51.5390261, -0.1425516],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.4974948, -0.1356583],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5886383, -0.0117625],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4097742, -0.2108084],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.538621, -0.1028346],
 [51.5886121, 0.0823982],
 [51.4974948, -0.1356583],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.6569225, -0.1949252],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.538621, -0.1028346],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5255162, 0.0352163],
 [51.4990805, -0.1938253],
 [51.4974948, -0.1356583],
 [51.4990156, -0.22915],
 [51.5672808, -0.2710568],
 [51.6522994, -0.0807119],
 [51.538621, -0.1028346],
 [51.5906113, -0.1109709],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.5132537, -0.3043136],
 [51.5202607, -0.0293396],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.538621, -0.1028346],
 [51.4935082, -0.1178424],
 [51.580559, -0.341995],
 [51.5351832, -0.4481378],
 [51.502781, -0.087738],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.4990805, -0.1938253],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5132537, -0.3043136],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5351832, -0.4481378],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.458373, -0.1891356],
 [51.6522994, -0.0807119],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.6522994, -0.0807119],
 [51.4974948, -0.1356583],
 [51.4611509, -0.0073177],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.502781, -0.087738],
 [51.41233, -0.300689],
 [51.5540666, 0.134017],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5351832, -0.4481378],
 [51.5202607, -0.0293396],
 [51.4990805, -0.1938253],
 [51.5672808, -0.2710568],
 [51.3769529, -0.0956895],
 [51.4990805, -0.1938253],
 [51.4990156, -0.22915],
 [51.4974948, -0.1356583],
 [51.5886121, 0.0823982],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5390261, -0.1425516],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5886121, 0.0823982],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.577924, 0.2120829],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.4990805, -0.1938253],
 [51.3769529, -0.0956895],
 [51.6522994, -0.0807119],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.6522994, -0.0807119],
 [51.4990156, -0.22915],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4990156, -0.22915],
 [51.5886383, -0.0117625],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.4990805, -0.1938253],
 [51.5202607, -0.0293396],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5390261, -0.1425516],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.5390261, -0.1425516],
 [51.458373, -0.1891356],
 [51.580559, -0.341995],
 [51.5255162, 0.0352163],
 [51.5906113, -0.1109709],
 [51.4990156, -0.22915],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4611509, -0.0073177],
 [51.3769529, -0.0956895],
 [51.4933675, 0.0098214],
 [51.577924, 0.2120829],
 [51.6522994, -0.0807119],
 [51.458373, -0.1891356],
 [51.4933675, 0.0098214],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.4935082, -0.1178424],
 [51.5436387, -0.0553621],
 [51.6522994, -0.0807119],
 [51.4933675, 0.0098214],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5351832, -0.4481378],
 [51.5436387, -0.0553621],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.461311, -0.303742],
 [51.6569225, -0.1949252],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4990805, -0.1938253],
 [51.5886121, 0.0823982],
 [51.5351832, -0.4481378],
 [51.6522994, -0.0807119],
 [51.5255162, 0.0352163],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.580559, -0.341995],
 [51.5202607, -0.0293396],
 [51.5886121, 0.0823982],
 [51.439933, 0.154327],
 [51.5436387, -0.0553621],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.5132537, -0.3043136],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.580559, -0.341995],
 [51.439933, 0.154327],
 [51.4611509, -0.0073177],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4097742, -0.2108084],
 [51.5351832, -0.4481378],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.4935082, -0.1178424],
 [51.461311, -0.303742],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.4935082, -0.1178424],
 [51.458373, -0.1891356],
 [51.439933, 0.154327],
 [51.580559, -0.341995],
 [51.4933675, 0.0098214],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.5436387, -0.0553621],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.4974948, -0.1356583],
 [51.5886383, -0.0117625],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.5436387, -0.0553621],
 [51.538621, -0.1028346],
 [51.439933, 0.154327],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5672808, -0.2710568],
 [51.4935082, -0.1178424],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.6569225, -0.1949252],
 [51.5202607, -0.0293396],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.3769529, -0.0956895],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.4609218, -0.373149],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5132537, -0.3043136],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.4097742, -0.2108084],
 [51.5436387, -0.0553621],
 [51.577924, 0.2120829],
 [51.4990156, -0.22915],
 [51.406025, 0.013156],
 [51.580559, -0.341995],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.4990156, -0.22915],
 [51.458373, -0.1891356],
 [51.5123443, -0.0909852],
 [51.4990156, -0.22915],
 [51.5906113, -0.1109709],
 [51.3769529, -0.0956895],
 [51.580559, -0.341995],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.3769529, -0.0956895],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.4609218, -0.373149],
 [51.3769529, -0.0956895],
 [51.538621, -0.1028346],
 [51.6569225, -0.1949252],
 [51.5351832, -0.4481378],
 [51.6569225, -0.1949252],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4974948, -0.1356583],
 [51.4609218, -0.373149],
 [51.6569225, -0.1949252],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5351832, -0.4481378],
 [51.5540666, 0.134017],
 [51.439933, 0.154327],
 [51.406025, 0.013156],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.5672808, -0.2710568],
 [51.458373, -0.1891356],
 [51.4974948, -0.1356583],
 [51.4974948, -0.1356583],
 [51.5906113, -0.1109709],
 [51.5906113, -0.1109709],
 [51.5540666, 0.134017],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.5390261, -0.1425516],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.6522994, -0.0807119],
 [51.5672808, -0.2710568],
 [51.5436387, -0.0553621],
 [51.502781, -0.087738],
 [51.461311, -0.303742],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.5886121, 0.0823982],
 [51.580559, -0.341995],
 [51.4097742, -0.2108084],
 [51.5886383, -0.0117625],
 [51.4611509, -0.0073177],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5390261, -0.1425516],
 [51.5132537, -0.3043136],
 [51.502781, -0.087738],
 [51.538621, -0.1028346],
 [51.4097742, -0.2108084],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.6569225, -0.1949252],
 [51.41233, -0.300689],
 [51.577924, 0.2120829],
 [51.3769529, -0.0956895],
 [51.461311, -0.303742],
 [51.458373, -0.1891356],
 [51.5202607, -0.0293396],
 [51.502781, -0.087738],
 [51.577924, 0.2120829],
 [51.5906113, -0.1109709],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.5540666, 0.134017],
 [51.577924, 0.2120829],
 [51.5132537, -0.3043136],
 [51.5436387, -0.0553621],
 [51.4933675, 0.0098214],
 [51.5886383, -0.0117625],
 [51.406025, 0.013156],
 [51.439933, 0.154327],
 [51.3769529, -0.0956895],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.4974948, -0.1356583],
 [51.461311, -0.303742],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4097742, -0.2108084],
 [51.458373, -0.1891356],
 [51.458373, -0.1891356],
 [51.4611509, -0.0073177],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.458373, -0.1891356],
 [51.4990805, -0.1938253],
 [51.6569225, -0.1949252],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5132537, -0.3043136],
 [51.5886383, -0.0117625],
 [51.458373, -0.1891356],
 [51.3769529, -0.0956895],
 [51.5390261, -0.1425516],
 [51.4990156, -0.22915],
 [51.3769529, -0.0956895],
 [51.5886121, 0.0823982],
 [51.406025, 0.013156],
 [51.5202607, -0.0293396],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.6569225, -0.1949252],
 [51.5390261, -0.1425516],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.461311, -0.303742],
 [51.5886121, 0.0823982],
 [51.6569225, -0.1949252],
 [51.580559, -0.341995],
 [51.5906113, -0.1109709],
 [51.4990805, -0.1938253],
 [51.461311, -0.303742],
 [51.580559, -0.341995],
 [51.4990156, -0.22915],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.5255162, 0.0352163],
 [51.5255162, 0.0352163],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.5390261, -0.1425516],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.4990156, -0.22915],
 [51.4609218, -0.373149],
 [51.4935082, -0.1178424],
 [51.5390261, -0.1425516],
 [51.461311, -0.303742],
 [51.5132537, -0.3043136],
 [51.461311, -0.303742],
 [51.3769529, -0.0956895],
 [51.5540666, 0.134017],
 [51.4933675, 0.0098214],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5906113, -0.1109709],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886121, 0.0823982],
 [51.458373, -0.1891356],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.6569225, -0.1949252],
 [51.5672808, -0.2710568],
 [51.5672808, -0.2710568],
 [51.538621, -0.1028346],
 [51.5540666, 0.134017],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.406025, 0.013156],
 [51.5132537, -0.3043136],
 [51.406025, 0.013156],
 [51.461311, -0.303742],
 [51.4933675, 0.0098214],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.461311, -0.303742],
 [51.538621, -0.1028346],
 [51.4990156, -0.22915],
 [51.4935082, -0.1178424],
 [51.3769529, -0.0956895],
 [51.4097742, -0.2108084],
 [51.41233, -0.300689],
 [51.406025, 0.013156],
 [51.5351832, -0.4481378],
 [51.5132537, -0.3043136],
 [51.538621, -0.1028346],
 [51.41233, -0.300689],
 [51.538621, -0.1028346],
 [51.4990805, -0.1938253],
 [51.5436387, -0.0553621],
 [51.4609218, -0.373149],
 [51.538621, -0.1028346],
 [51.4974948, -0.1356583],
 [51.4935082, -0.1178424],
 [51.4609218, -0.373149],
 [51.458373, -0.1891356],
 [51.5255162, 0.0352163],
 [51.3769529, -0.0956895],
 [51.4609218, -0.373149],
 [51.4974948, -0.1356583],
 [51.538621, -0.1028346],
 [51.538621, -0.1028346],
 [51.458373, -0.1891356],
 [51.4609218, -0.373149],
 [51.461311, -0.303742],
 [51.5202607, -0.0293396],
 [51.458373, -0.1891356],
 [51.4935082, -0.1178424],
 [51.4933675, 0.0098214],
 [51.3769529, -0.0956895],
 [51.4990156, -0.22915],
 [51.5390261, -0.1425516],
 [51.4097742, -0.2108084],
 [51.4990805, -0.1938253],
 [51.4097742, -0.2108084],
 [51.5202607, -0.0293396],
 [51.580559, -0.341995],
 [51.577924, 0.2120829],
 [51.4974948, -0.1356583],
 [51.406025, 0.013156],
 [51.5886121, 0.0823982],
 [51.5436387, -0.0553621],
 [51.5351832, -0.4481378],
 [51.580559, -0.341995],
 [51.4974948, -0.1356583],
 [51.502781, -0.087738],
 [51.4609218, -0.373149],
 [51.5255162, 0.0352163],
 [51.580559, -0.341995],
 [51.5886383, -0.0117625],
 [51.4990805, -0.1938253],
 [51.502781, -0.087738],
 [51.3769529, -0.0956895],
 [51.577924, 0.2120829],
 [51.577924, 0.2120829],
 [51.406025, 0.013156],
 [51.4990156, -0.22915],
 [51.538621, -0.1028346],
 [51.4609218, -0.373149],
 [51.406025, 0.013156],
 [51.5390261, -0.1425516],
 [51.4990805, -0.1938253],
 [51.4990805, -0.1938253],
 [51.4935082, -0.1178424],
 [51.406025, 0.013156],
 [51.4933675, 0.0098214],
 [51.458373, -0.1891356],
 [51.6569225, -0.1949252],
 [51.458373, -0.1891356],
 [51.577924, 0.2120829],
 [51.502781, -0.087738],
 [51.5886121, 0.0823982],
 [51.4990156, -0.22915],
 [51.5886121, 0.0823982],
 [51.577924, 0.2120829],
 [51.4611509, -0.0073177],
 [51.4611509, -0.0073177]]

In [189]:
# Distance to the nearest outstanding school feature 
# this code can be placed in the while loop constructing all of the houses and their features

lat_example = houses['59, Bateman Close, Barking, Greater London IG11 8QS']["latitude"]
lon_example = houses['59, Bateman Close, Barking, Greater London IG11 8QS']["longitude"]

coords_1 = (lat_example, lon_example)
lower_boundary, upper_boundary = -0.05, 0.05
school_distances = []
for i in outstanding_schools:
    if (i[0] - lat_example > upper_boundary) or i[0] - lat_example < lower_boundary:
        continue
    elif (i[1] - lon_example > upper_boundary) or i[1] - lon_example < lower_boundary:
        continue
    else:
        coords_2 = (i[0], i[1])
        school_distances.append(geopy.distance.geodesic(coords_1, coords_2).km)

min(school_distances)


3.7150173193710945

In [195]:
# Approach to help find the prices of nearby, similar properties
# THIS CODE NEEDS TO BE PERFORMED AFTER ALL OF THE HOUSES ARE STORED
# or even you could it after each URL iteration of the local area URL's, to save looping through all of the houses

######################## DO NOT RUN THIS CODE ##################


for i, v in houses.items():
    local_similar_prices = []
    # need to store the postcode, each house has a postcode so this will be easier
    postcode = i[-8:]
    if postcode[0] == " ":
        postcode = i[-7:]
    postcode_area = postcode.split(" ")[0]
    prop_type = v["property_type"]
    
    for a, b in houses.items():
        postcode = i[-8:]
        if postcode[0] == " ":
            postcode = i[-7:]
        postcode_area1 = postcode.split(" ")[0]
        if postcode_area == postcode_area1:
            if b["property_type"] == prop_type:
                local_similar_prices.append(b['price'])
    
    avg_local_price = avg(local_similar_prices)
    v["avg_local_price"] = avg_local_price
    
####################################################################

RM9
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG11
IG11
IG1
IG11
RM9
RM9
IG11
IG11
IG11
RM9
IG11
IG11
IG11
RM9
RM9
IG11
IG3
IG11
IG11
IG1
RM8
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG11
RM8
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM8
IG11
IG11
IG11
IG11
IG11
IG11
RM9
RM9
IG11
IG11
RM9
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG11
IG11
IG11
IG3
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG1
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG11
IG11
IG11
IG11
RM9
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG3
IG11
IG11
IG11
IG11
RM8
IG11
IG11
IG11
IG11
IG11
IG11
IG3
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG3
RM9
IG11
IG11
IG11
IG11
IG3
IG11
IG11
IG11
RM9
IG11
IG11
IG11
IG11
RM9
RM9
IG11
IG11
IG3
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
IG11
IG11
IG11
IG11
IG11
IG11
IG11
IG11
RM9
RM8
RM9
IG11
IG11
RM